In [1]:
import numpy as np
import pandas as pd

In [2]:
# 要数据合并的文件名
filenames = ['2018年灌溉用水量（IWU）(grids).csv', '2018年降水量(grids).csv', '2018年潜在蒸散(grids).csv']

In [3]:
# 读取文件数据
df1 = pd.read_csv(filenames[0])
df2 = pd.read_csv(filenames[1])
df3 = pd.read_csv(filenames[2])

In [4]:
#同一单位：灌溉用水和降水量单位都是mm/month,潜在蒸散单位是mm/day
df3 = df3.set_index('grid_num').multiply(30).reset_index() 

In [5]:
# 按 grid_num 合并 df2（降水量）到 df1
df1 = df1.merge(
    df2[['grid_num'] + [f'{m}月' for m in range(1, 13)]],
    on='grid_num', 
    how='left',
    suffixes=('', '_df2')  # 避免列名冲突（若需保留原 df1 列可省略）
)

# 关键步骤：将合并后 df2 相关列的 NaN 替换为 0
for month in [f'{m}月_df2' for m in range(1, 13)]:
    df1.fillna({month:0}, inplace=True)  # NaN 填充为 0

In [6]:
# 按 grid_num 合并 df3（潜在蒸散）到 df1
df1 = df1.merge(
    df3[['grid_num'] + [f'{m}月' for m in range(1, 13)]],
    on='grid_num', 
    how='left',
    suffixes=('', '_df3')
)

# 关键步骤：将合并后 df3 相关列的 NaN 替换为 0
for month in [f'{m}月_df3' for m in range(1, 13)]:
    df1.fillna({month:0}, inplace=True)  # NaN 填充为 0

In [7]:
# 遍历月份列，将合并的 df2、df3 数据加到 df1 原始列
for month in [f'{m}月' for m in range(1, 13)]:
    # 处理 df2 合并的列（如 '1月_df2'）
    df1[month] += df1.get(f'{month}_df2', 0)
    # 处理 df3 合并的列（如 '1月_df3'）
    # df1[month] -= df1.get(f'{month}_df3', 0)  
    # 删除临时合并列（可选，清理冗余）
    df1.drop(columns=[f'{month}_df2', f'{month}_df3'], inplace=True, errors='ignore')

In [8]:
# 存储结果
df1.round(2).to_csv("./2018年水输入总量(汇总对照表).csv", index=False)

In [9]:
# def get_lat_lon(df, grid_num):
    
#     lats, lons = [], []
    
#     for k in df.index:
#         lat = -90+(df.loc[k,'grid_num']//720)*0.5+0.25
#         lon = -180+(df.loc[k,'grid_num']%720)*0.5-0.25
#         lats.append(lat)
#         lons.append(lon)

#     return(lats, lons)

In [10]:
# lat, lon = get_lat_lon(df1, 'grid_num')
# df_result = pd.DataFrame({'grid_num': df1['grid_num'].values, 'lat_center':lat, 'lon_center':lon})
# df_result = df_result.merge(df1, on='grid_num', how='left')

In [11]:
# # 存储结果
# df_result.to_csv("./2018年水输入总量(汇总).csv", index=False)

In [12]:
# df_result[df_result[df_result.columns[1:]].values>0]